# Import

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import os
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv() 
DATASET_FOLDER = os.getenv('DATASET_FOLDER')

In [4]:
raw_df = pd.read_csv(f"{DATASET_FOLDER}/Food_Establishment_Inspection_Data.csv")

/tmp/ipykernel_11944/3624628935.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(f"{DATASET_FOLDER}/Food_Establishment_Inspection_Data.csv")


# Data Wrangling

In [5]:
cols = { k:k.replace(' ','_').lower() for k in raw_df.keys()}
cols
pre_df = raw_df.rename(columns=cols)

In [6]:
# pre_df = pre_df.dropna()

In [7]:
pre_df.city=pre_df.city.str.upper()

In [8]:
pre_df.drop(['program_identifier','inspection_business_name','inspection_serial_num','violation_record_id','business_id'], axis=1);

In [9]:
pre_df["inspection_date"]=pd.to_datetime(pre_df["inspection_date"])

pre_df['weekday'] = pre_df.inspection_date.dt.day_name()


In [10]:
df = pre_df.reindex()

In [11]:
# Fix wrong longitude 
df.loc[df['longitude'] > 122, 'longitude'] *=-1

In [12]:
uniques = {}
uniques_count={}
df.apply(
    (
        lambda x: (uniques.update(
            {x.name:x.unique()}
        ),
          uniques_count.update(
              {x.name:len(x.unique())} 
          ))
    ),
    axis=0)
df_uniques = pd.DataFrame.from_dict(uniques_count, orient='index', columns=['Count'])
df_uniques = df_uniques.sort_values(by='Count', ascending=False)
# df_uniques

## deleting useless col


In [13]:

not_valid_inspection_result_rows=pre_df[(pre_df.inspection_result !='Unsatisfactory') & (pre_df.inspection_result !='Complete') &
(pre_df.inspection_result !='Incomplete') & (pre_df.inspection_result !='Satisfactory')
]
pre_df.drop(not_valid_inspection_result_rows.index, inplace=True)


In [14]:
df = df.drop('zip_code',axis=1)
#same name
df = df.drop(['program_identifier','inspection_business_name'],axis=1)
#low human readability
df = df.drop(['business_id','inspection_serial_num','violation_record_id'],axis=1)


In [15]:
# relation = df[['name','inspection_date','description','inspection_type','inspection_score','inspection_result','inspection_closed_business','violation_type','violation_description','violation_points','grade']]

In [16]:
# relation['violation_description'].unique()

In [17]:
df.drop(not_valid_inspection_result_rows.index, inplace=True)

In [18]:
# relation[['inspection_result','violation_description']].head(50)

In [19]:
# relation['inspection_result'].unique()

In [20]:
# relation[['name','inspection_date','violation_points','inspection_score','grade','description']]

In [21]:
df['risk_category'] = df['description'].apply(lambda x: str(re.findall(r'Risk.*',x)).replace('[','').replace(']','').replace("'",''))

In [22]:
df['description'] = df['description'].apply(lambda x: str(re.findall(r'(.*)Risk.*',x)).replace('[','').replace(']','').replace("'",'')[:-2])

In [23]:
df['violation_description_code'] = df['violation_description'].apply(lambda x: re.findall(r'(\d*).*',str(x))[0])

In [24]:
df['violation_description_type'] = df['violation_description'].apply(lambda x: re.findall(r'\d*(.*)',str(x))[0].replace("-","").strip())

In [25]:
df['risk_category'].unique()

array(['Risk Category III', 'Risk Category II', 'Risk Category I', '',
       'Risk II'], dtype=object)

In [26]:
def casting(x):
    if x == 'Risk II':
        return 2
    elif x == 'Risk Category I':
        return 1
    elif x == 'Risk Category II':
        return 2
    elif x == 'Risk Category III':
        return 3
    else:
        return None

In [27]:
df['risk_category'] = df['risk_category'].apply(casting)

In [28]:
df['risk_category'].unique()

array([ 3.,  2.,  1., nan])

In [29]:
new_df = df[['weekday',
          'name',
          'description',
          'risk_category',
          'city',
          'address',
          'longitude',
          'latitude',
          'inspection_type',
          'inspection_score',
          'violation_points',
          'grade',
          'inspection_result',
          'inspection_closed_business',
          'violation_type',
          'violation_description_code',
          'violation_description_type']]

In [30]:
new_df.set_index(df['inspection_date'],inplace=True)

In [31]:
new_df.index.is_unique

False

In [32]:
new_df[(new_df['inspection_closed_business'] == True) & (new_df['violation_type'] == 'RED')]['name'].unique()

array(['13 COINS', "AL'S GOURMET SAUSAGE #5", 'ART MARBLE 21',
       'ASIA BBQ & FAST FOOD', 'BEEZNEEZ GOURMET SAUSAGE (KC287)',
       'BEST CORN #1', 'BLACKSTAR KEBAB (KC938)', 'BOMBAY GRILL',
       'BOUMBA HOTDOG', 'BROS TERIYAKI', 'CALLED A CHICKEN',
       "CARMELO'S TACOS CHERRY ST", 'CEDARS BROOKLYN DELI',
       'CHIPOTLE MEXICAN GRILL #2228', 'CHOPSTICKS CUISINE',
       'CHU MINH TOFU & VEGETARIAN DELI', "CHUCK E CHEESE'S #305",
       'COURTYARD BY MARRIOTT - STARBUCKS', 'CRAVE BY SUITE J (KC332)',
       'CRAWFISH HOUSE', 'DIRTY DOG (KC299)', 'DOG IN THE PARK',
       'DONG TING CHUN', 'EL CAMION', 'FEAST', 'FOODY MOODY',
       'FREMONT BOWL', 'GOLDEN DAISY RESTAURANT', 'GREAT WESTERN PACIFIC',
       'GRILL CITY', 'HABIT BURGER GRILL, THE', 'HALLAVA FALAFEL',
       'HANAHREUM MART', 'HARBOR CITY RESTAURANT', 'HONG KONG BISTRO',
       'IKIIKI', 'INDIAN CURRY PALACE', 'KFC #327', 'KING BUFFET',
       'KURA REVOLVING SUSHI BAR BELLEVUE', 'LA PINA/TRES HERMANOS',
       

In [42]:
list_of_closed_business = new_df[(new_df['inspection_closed_business'] == True)]['name'].unique()

In [43]:
for i in list_of_closed_business:
    print(i)

13 COINS
7-ELEVEN STORE #2361-27283C
AL'S GOURMET SAUSAGE #5
ART MARBLE 21
ASIA BBQ & FAST FOOD
BEEZNEEZ GOURMET SAUSAGE (KC287)
BEST CORN #1
BLACKSTAR KEBAB (KC938)
BOMBAY GRILL
BOUMBA HOTDOG
BROS TERIYAKI
CALLED A CHICKEN
CARMELO'S TACOS CHERRY ST
CEDARS BROOKLYN DELI
CHINESE SAUERKRAUT FISH
CHIPOTLE MEXICAN GRILL #2228
CHOPSTICKS CUISINE
CHU MINH TOFU & VEGETARIAN DELI
CHUCK E CHEESE'S #305
CHURCH'S CHICKEN
COURTYARD BY MARRIOTT - STARBUCKS
CRAVE BY SUITE J (KC332)
CRAWFISH HOUSE
DIRTY DOG (KC299)
DOG IN THE PARK
DONG TING CHUN
EL CAMION
FEAST
FOODY MOODY
FREMONT BOWL
GERALDINE'S COUNTER
GOLDEN DAISY RESTAURANT
GREAT WESTERN PACIFIC
GRILL CITY
HABIT BURGER
HABIT BURGER GRILL, THE
HALLAVA FALAFEL
HANAHREUM MART
HARBOR CITY RESTAURANT
HIGHLAND CHEVRON
HONG KONG BISTRO
IKIIKI
INDIAN CURRY PALACE
KFC #327
KING BUFFET
KURA REVOLVING SUSHI BAR BELLEVUE
LA PINA/TRES HERMANOS
LADYBUG ESPRESSO
LARI ADDA  - KC875
LUNCHBOX LABORATORY
MAHARAJA CUSINE OF INDIA
MANCHU WOK AT SEA-TAC
MAYURI FOODS 

In [44]:
len(list_of_closed_business)

120

In [45]:
df_closed_business_by_name = new_df[new_df['name'].apply(lambda x : x in list_of_closed_business)]


we didnt have any record in https://kingcounty.gov/en/legacy/depts/health/environmental-health/food-safety/inspection-system/closures for sample that get 0 inspection score

In [63]:
df_closed_business_by_name[(df_closed_business_by_name['inspection_closed_business'] == True) & (df_closed_business_by_name['inspection_score'] == 0)].sort_index()

,weekday,name,description,risk_category,city,address,longitude,latitude,inspection_type,inspection_score,violation_points,grade,inspection_result,inspection_closed_business,violation_type,violation_description_code,violation_description_type
inspection_date,,,,,,,,,,,,,,,,,
2007-07-23,Monday,PIONEER GRILL@VARAMINI COMMISSARY,Mobile Food Unit,2.0,SEATTLE,904 OCCIDENTAL AVE S,-122.333779,47.586432,Routine Inspection/Field Review,0.0,0,NaN,Unsatisfactory,True,NaN,,nan
2008-08-29,Friday,QUARTER CHUTE CAFE,Seating 51-150,3.0,AUBURN,2828 EMERALD DOWNS DR,-122.236430,47.332584,Consultation/Education - Field,0.0,0,1.0,Complete,True,NaN,,nan
2009-06-17,Wednesday,HANAHREUM MART,Meat/Sea Food,3.0,FEDERAL WAY,31217 PACIFIC HWY S,-122.315255,47.321967,Routine Inspection/Field Review,0.0,0,NaN,Unsatisfactory,True,NaN,,nan
2009-06-17,Wednesday,HANAHREUM MART,Seating 0-12,3.0,FEDERAL WAY,31217 PACIFIC HWY S,-122.315255,47.321967,Routine Inspection/Field Review,0.0,0,1.0,Unsatisfactory,True,NaN,,nan
2009-06-17,Wednesday,HANAHREUM MART,Meat/Sea Food,3.0,FEDERAL WAY,31217 PACIFIC HWY S,-122.315255,47.321967,Routine Inspection/Field Review,0.0,0,NaN,Unsatisfactory,True,NaN,,nan
2009-06-18,Thursday,HANAHREUM MART,Meat/Sea Food,3.0,FEDERAL WAY,31217 PACIFIC HWY S,-122.315255,47.321967,Return Inspection,0.0,0,NaN,Unsatisfactory,True,NaN,,nan
2011-04-22,Friday,SANDHU SHELL MINI-MART,Seating 0-12,3.0,BELLEVUE,2659 148TH AVE SE,-122.143108,47.586219,Return Inspection,0.0,0,1.0,Unsatisfactory,True,NaN,,nan
2012-03-21,Wednesday,PING'S FOOD MART,Seating 13-50,3.0,SEATTLE,508 S KING ST,-122.327295,47.598483,Return Inspection,0.0,0,1.0,Unsatisfactory,True,NaN,,nan
2014-10-13,Monday,7-ELEVEN STORE #2361-27283C,Seating 0-12,3.0,DES MOINES,820 KENT DES MOINES RD,-122.323365,47.397102,Routine Inspection/Field Review,0.0,0,2.0,Unsatisfactory,True,NaN,,nan


In [47]:
df_closed_business_by_name[(df_closed_business_by_name['inspection_closed_business'] == True) & (df_closed_business_by_name['inspection_score'] > 0)].sort_index().name.unique()

108

In [48]:
df_closed_business_by_name[(df_closed_business_by_name['inspection_closed_business'] == True) ].sort_index()['violation_description_code'].value_counts()

violation_description_code
        107
0600     76
0400     51
2110     43
1600     37
1900     36
0100     31
1400     28
3300     26
3400     25
0200     22
4200     20
2600     20
1710     20
4100     18
3200     17
2500     17
4300     16
4400     16
2200     15
4000     12
1300     12
3000     11
0500      9
1000      9
4800      8
0900      8
3700      7
3800      6
5000      6
2120      5
4900      5
3100      4
0700      4
4700      4
1100      4
2000      4
1500      3
0300      3
2300      3
1200      3
2900      3
2800      3
3900      2
4500      2
1800      2
0800      2
3600      1
4600      1
2700      1
Name: count, dtype: int64

In [49]:
inspection_score_graterthan_zero_and_close = df_closed_business_by_name[(df_closed_business_by_name['inspection_closed_business'] == True) & (df_closed_business_by_name['inspection_score'] > 0)]

In [52]:
inspection_score_graterthan_zero_and_close.sort_index()

,weekday,name,description,risk_category,city,address,longitude,latitude,inspection_type,inspection_score,violation_points,grade,inspection_result,inspection_closed_business,violation_type,violation_description_code,violation_description_type
inspection_date,,,,,,,,,,,,,,,,,
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,15,NaN,Unsatisfactory,True,RED,,Hands washed as required
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,10,NaN,Unsatisfactory,True,RED,,Adequate handwashing facilities
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,5,NaN,Unsatisfactory,True,RED,,Accurate thermometer provided and used ...
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,25,NaN,Unsatisfactory,True,RED,,No room temperature storage; proper use of tim...
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,10,NaN,Unsatisfactory,True,RED,,"Food in good condition, safe and unadulterat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-15,Tuesday,RANCHO BRAVO TACOS,Seating 13-50,3.0,SEATTLE,1001 E PINE ST,-122.318983,47.615101,Routine Inspection/Field Review,70.0,10,4.0,Unsatisfactory,True,RED,2110,Proper cold holding temperatures (greater than...
2023-08-15,Tuesday,RANCHO BRAVO TACOS,Seating 13-50,3.0,SEATTLE,1001 E PINE ST,-122.318983,47.615101,Routine Inspection/Field Review,70.0,25,4.0,Unsatisfactory,True,RED,1600,Proper cooling procedure
2023-08-30,Wednesday,"HABIT BURGER GRILL, THE",Seating 51-150,3.0,ISSAQUAH,1676 9TH AVE NE STE 120,-122.019840,47.544010,Routine Inspection/Field Review,38.0,10,4.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities


In [54]:
inspection_score_graterthan_zero_and_close[inspection_score_graterthan_zero_and_close['violation_description_code'] == '0600']

,weekday,name,description,risk_category,city,address,longitude,latitude,inspection_type,inspection_score,violation_points,grade,inspection_result,inspection_closed_business,violation_type,violation_description_code,violation_description_type
inspection_date,,,,,,,,,,,,,,,,,
2014-04-28,Monday,13 COINS,Seating 51-150,3.0,SEATTLE,18000 PACIFIC HWY S,-122.295768,47.441590,Routine Inspection/Field Review,100.0,10,1.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2015-07-27,Monday,ASIA BBQ & FAST FOOD,Seating 0-12,3.0,SEATTLE,655A S JACKSON ST,-122.324691,47.599019,Routine Inspection/Field Review,125.0,10,2.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2016-03-17,Thursday,BEEZNEEZ GOURMET SAUSAGE (KC287),Mobile Food Unit,2.0,RENTON,960 B HARRINGTON AVE NE,-122.387002,47.668390,Routine Inspection/Field Review,10.0,10,NaN,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2012-09-13,Thursday,BEST CORN #1,Mobile Food Unit,1.0,SEATTLE,9811 15th AVE SW,-122.354232,47.515381,Routine Inspection/Field Review,25.0,10,NaN,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2023-06-17,Saturday,BLACKSTAR KEBAB (KC938),Mobile Food Unit,3.0,TUKWILA,10836 E MARGINAL WAY S,-122.344159,47.683113,Routine Inspection/Field Review,85.0,10,NaN,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-25,Wednesday,WALAALAHA MINI MARKET,Seating 0-12,3.0,SEATTLE,8318 RAINIER AVE S,-122.269775,47.528974,Return Inspection,95.0,10,2.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2022-02-01,Tuesday,WET MEX,Mobile Food Unit,3.0,SHORELINE,14602 15TH AVE NE,-122.312436,47.735039,Routine Inspection/Field Review,15.0,10,NaN,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
2016-07-11,Monday,YOKO 3,Seating 13-50,3.0,ISSAQUAH,4516 KLAHANIE DR SE,-122.005773,47.565777,Routine Inspection/Field Review,141.0,10,1.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities


In [59]:
inspection_score_graterthan_zero_and_close.sort_index().loc

,weekday,name,description,risk_category,city,address,longitude,latitude,inspection_type,inspection_score,violation_points,grade,inspection_result,inspection_closed_business,violation_type,violation_description_code,violation_description_type
inspection_date,,,,,,,,,,,,,,,,,
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,15,NaN,Unsatisfactory,True,RED,,Hands washed as required
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,10,NaN,Unsatisfactory,True,RED,,Adequate handwashing facilities
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,5,NaN,Unsatisfactory,True,RED,,Accurate thermometer provided and used ...
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,25,NaN,Unsatisfactory,True,RED,,No room temperature storage; proper use of tim...
2006-01-09,Monday,TAQUERIA GUADALAJARA,Mobile Food Unit,3.0,NEWCASTLE,13256 NEWCASTLE COMMONS,-122.141635,47.648526,Routine Inspection/Field Review,90.0,10,NaN,Unsatisfactory,True,RED,,"Food in good condition, safe and unadulterat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-15,Tuesday,RANCHO BRAVO TACOS,Seating 13-50,3.0,SEATTLE,1001 E PINE ST,-122.318983,47.615101,Routine Inspection/Field Review,70.0,10,4.0,Unsatisfactory,True,RED,2110,Proper cold holding temperatures (greater than...
2023-08-15,Tuesday,RANCHO BRAVO TACOS,Seating 13-50,3.0,SEATTLE,1001 E PINE ST,-122.318983,47.615101,Routine Inspection/Field Review,70.0,25,4.0,Unsatisfactory,True,RED,1600,Proper cooling procedure
2023-08-30,Wednesday,"HABIT BURGER GRILL, THE",Seating 51-150,3.0,ISSAQUAH,1676 9TH AVE NE STE 120,-122.019840,47.544010,Routine Inspection/Field Review,38.0,10,4.0,Unsatisfactory,True,RED,0600,Adequate handwashing facilities
